In [65]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
import math
import numpy as np
from sklearn.metrics import r2_score
from utils.plot_data import plot_sensor_measurement, plot_wind_rose, rmse, calc_r2


start_date = datetime(2023,12, 22, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 22, 23, 59, 59).replace(tzinfo=timezone.utc)

filter = '1h'

#sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20]
sensor_id = [10,12,16,18,20]

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")).filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()


# Plot other data directly from local database measurements.parquet

In [66]:
plot_sensor_measurement(df,sensor_id=sensor_id,col_name="gmp343_raw",filter=filter)

In [67]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_compensated",filter=filter)

In [68]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_filtered",filter=filter)

In [69]:
plot_sensor_measurement(df,sensor_id=[1,2,4,5,6,7,8,14,15],col_name="wxt532_temperature",filter=filter)

In [70]:
# plot wind at 3m pole inlet
plot_wind_rose(df,6, 'TUM')
# Maisach
plot_wind_rose(df,1, 'Maisach')
# Rechts-der-Isar
plot_wind_rose(df,2, 'Rechts-der-Isar')
# Finsing
plot_wind_rose(df,15, 'Finsing')
# Großhadern
plot_wind_rose(df,4, 'Großhadern')
# Großhadern
plot_wind_rose(df,5, 'Pasing')
# Feldkirchen
plot_wind_rose(df,7, 'Feldkirchen')
# Taufkirchen
plot_wind_rose(df,8, 'Taufkirchen')
# Finsing
plot_wind_rose(df,14, 'DLR')

In [71]:
plot_sensor_measurement(df,sensor_id,col_name="wxt532_speed_avg",filter=filter)

In [72]:
plot_sensor_measurement(df,sensor_id,col_name="wxt532_direction_avg",filter=filter)

In [73]:
# Plot auxilliary data
plot_sensor_measurement(df,sensor_id,col_name="sht45_humidity",filter=filter)

In [74]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_humidity",filter=filter)

In [75]:
plot_sensor_measurement(df,sensor_id,col_name="sht45_temperature",filter=filter)

In [76]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_temperature",filter=filter)

In [77]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_pressure",filter=filter)

In [78]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_temperature",filter=filter)

In [79]:
plot_sensor_measurement(df,sensor_id,col_name="raspi_cpu_usage",filter=filter)

In [80]:
plot_sensor_measurement(df,sensor_id,col_name="raspi_memory_usage",filter=filter)


In [81]:
plot_sensor_measurement(df,sensor_id,col_name="raspi_disk_usage",filter=filter)


In [82]:
plot_sensor_measurement(df,sensor_id,col_name="raspi_cpu_temperature",filter=filter)

In [83]:
plot_sensor_measurement(df,sensor_id,col_name="enclosure_bme280_humidity",filter=filter)

In [84]:
plot_sensor_measurement(df,sensor_id,col_name="enclosure_bme280_pressure",filter=filter)

In [85]:
plot_sensor_measurement(df,sensor_id,col_name="enclosure_bme280_temperature",filter=filter)

# Calibration Data

In [86]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bottle_id",filter= '30s')

In [87]:
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_filtered",filter= '30s')

In [88]:
plot_sensor_measurement(df,sensor_id,col_name="cal_sht45_humidity",filter= '10s')

In [89]:
plot_sensor_measurement(df,sensor_id,col_name="cal_sht45_temperature",filter= '30s')

In [90]:
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_temperature",filter= '30s')

In [91]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bme280_temperature",filter= '30s')

In [92]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bme280_humidity",filter= '30s')

In [93]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bme280_pressure",filter= '30s')

In [94]:
# Plot 400ppm Calibration Bottle 
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_filtered",filter= '1m',cut_above= 450,cut_below=350)

In [96]:
# Plot 800ppm Calibration Bottle 
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_filtered",filter= '30s',cut_below=460)